In [3]:
import pandas as pd

df = pd.read_csv('dataset/synthetic_logs.csv')
df.head(3)

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [4]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [5]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

The user wants to generate text embeddings for the `log_message` column using the `sentence-transformers` library. Based on the context of the previous request (clustering with DBSCAN), the goal is to use these semantic embeddings as input for the DBSCAN clustering algorithm instead of a simple frequency-based approach like TF-IDF.

Here is the plan:
1.  Install `sentence-transformers` and `scikit-learn`.
2.  Load a pre-trained Sentence Transformer model (e.g., `all-MiniLM-L6-v2` which is fast and effective).
3.  Generate embeddings for the log messages.
4.  Cluster these embeddings using DBSCAN with a metric suitable for embeddings (like cosine distance).



In [35]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['log_message'].tolist())
embeddings[:2]

array([[-1.02939673e-01,  3.35458741e-02, -2.20260676e-02,
         1.55104266e-03, -9.86922532e-03, -1.78956285e-01,
        -6.34410158e-02, -6.01762086e-02,  2.81108543e-02,
         5.99619895e-02, -1.72618385e-02,  1.43369357e-03,
        -1.49560049e-01,  3.15287942e-03, -5.66030405e-02,
         2.71685142e-02, -1.49890827e-02, -3.54037359e-02,
        -3.62936184e-02, -1.45410867e-02, -5.61493495e-03,
         8.75538886e-02,  4.55120392e-02,  2.50963718e-02,
         1.00187194e-02,  1.24266790e-02, -1.39923587e-01,
         7.68696144e-02,  3.14095393e-02, -4.15245071e-03,
         4.36902530e-02,  1.71249788e-02, -8.00950751e-02,
         5.74005730e-02,  1.89092103e-02,  8.55261907e-02,
         3.96399423e-02, -1.34371817e-01, -1.44361309e-03,
         3.06702894e-03,  1.76854089e-01,  4.44887718e-03,
        -1.69274695e-02,  2.24266443e-02, -4.35050502e-02,
         6.09023077e-03, -9.98169743e-03, -6.23972639e-02,
         1.07372375e-02, -6.04898110e-03, -7.14660957e-0

In [36]:
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

df['cluster'] = clusters
df.head()

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [37]:
df[df.cluster==1]


,timestamp,source,log_message,target_label,complexity,cluster,regex_label
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1,None
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1,None
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1,None
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1,None
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1,None
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,bert,1,None
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,bert,1,None
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,bert,1,None
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,bert,1,None


In [38]:
cluster_counts = df['cluster'].value_counts().sort_values(ascending=False)

for cluster_id, count in cluster_counts.items():
    if count > 10:
        print(f"Cluster {cluster_id}: {count} records")

        sample_logs = df[df['cluster'] == cluster_id]['log_message'].head(5).tolist()

        for log in sample_logs:
            print(f"- {log[:150]}...")
        print("-" * 40)


Cluster 0: 1017 records
- nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.1...
- nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.1...
- nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.1...
- nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.1...
- nova.osapi_compute.wsgi.server [req-2bf7cfee-a236-42f3-8fb1-96fefab0b302 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.1...
----------------------------------------
Cluster 5: 147 records
- nova.compute.claims [req-a07ac654-8e81-416d-bfbb-189116b07969 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - 

In [39]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for regex, label in regex_patterns.items():
        if re.search(regex, log_message, re.IGNORECASE):
            return label
    return None

In [40]:
classify_with_regex("User User494 logged OuT.")

'User Action'

In [41]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [42]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


In [43]:
target_label_counts = df_non_regex['target_label'].value_counts()

print(target_label_counts[target_label_counts <= 5])


target_label
Workflow Error         4
Deprecation Warning    3
Name: count, dtype: int64


In [44]:
df_non_legacy = df_non_regex[df_non_regex.source != 'LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [45]:
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
embeddings[:2]

array([[-1.02939673e-01,  3.35458741e-02, -2.20260676e-02,
         1.55104266e-03, -9.86922532e-03, -1.78956285e-01,
        -6.34410158e-02, -6.01762086e-02,  2.81108543e-02,
         5.99619895e-02, -1.72618385e-02,  1.43369357e-03,
        -1.49560049e-01,  3.15287942e-03, -5.66030405e-02,
         2.71685142e-02, -1.49890827e-02, -3.54037359e-02,
        -3.62936184e-02, -1.45410867e-02, -5.61493495e-03,
         8.75538886e-02,  4.55120392e-02,  2.50963718e-02,
         1.00187194e-02,  1.24266790e-02, -1.39923587e-01,
         7.68696144e-02,  3.14095393e-02, -4.15245071e-03,
         4.36902530e-02,  1.71249788e-02, -8.00950751e-02,
         5.74005730e-02,  1.89092103e-02,  8.55261907e-02,
         3.96399423e-02, -1.34371817e-01, -1.44361309e-03,
         3.06702894e-03,  1.76854089e-01,  4.44887718e-03,
        -1.69274695e-02,  2.24266443e-02, -4.35050502e-02,
         6.09023077e-03, -9.98169743e-03, -6.23972639e-02,
         1.07372375e-02, -6.04898110e-03, -7.14660957e-0

In [47]:
X = filtered_embeddings
y = df_non_legacy['target_label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [49]:
import joblib

joblib.dump(clf, '../models/log_classifier.joblib')

['models/log_classifier.joblib']